In [2]:
import os
import ast
import sys
import json
import yaml
import re
from json import JSONDecodeError
from concurrent.futures import ThreadPoolExecutor, as_completed
from datetime import datetime
from pymongo import MongoClient


import gradio as gr
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, fcluster

# Import libraries for working with language models and Google Gemini
from langchain_core.prompts import PromptTemplate
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate

# Read configuration
config_file = '../gradio_config.yaml'
with open(config_file, 'r') as fin:
    config = yaml.safe_load(fin)
# end with
groq_api_key = os.environ['GROQ_API_KEY']
GEMINI_KEY   = os.environ['GEMINI_KEY']
MONGO_URI    = os.environ['MONGO_URI']
genai.configure(api_key=GEMINI_KEY)

# Initialise mongo client
mongo_client = MongoClient(MONGO_URI)

# Setup default LLM model
default_llm = genai.GenerativeModel('gemini-1.5-flash-latest')

In [7]:
def load_database():
    # Connect to the MongoDB client
    try:
        db = mongo_client[config["database"]["name"]]
        train_documents = db[config["database"]["train_collection"]].find()
        print("Train data successfully fetched from MongoDB\n")
    except Exception as error: 
        print(f"Unable to fetch train data from MongoDB. Check your connection the database...\nERROR: {error}\n")
        sys.exit()   
    try:
        test_docs = db[config["database"]["test_collection"]].find()
        print("Test data successfully fetched from MongoDB\n")
    except:
        print(f"Unable to fetch test data from MongoDB. Check your connection the database...\nERROR: {error}\n")
        sys.exit()
    df_train = pd.DataFrame.from_dict(list(train_documents))
    df_test = pd.DataFrame.from_dict(list(test_docs))
    df = pd.concat([df_train, df_test], axis=0)
    return df

def clean_llm_score(output):
    text = output.parts[0].text.replace("```", '').replace('json','')
    result = json.loads(text)
    return result

def clean_llm_output(output):
        text = output.parts[0].text.replace("```", '').replace('json','')
        result = json.loads(text)
        return result

def format_timeline_date(date_str):
    formats = ['%Y', '%Y-%m-%d', '%Y-%m']
    for fmt in formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            if fmt == '%Y':
                return date_obj.strftime('%Y')
            elif fmt == '%Y-%m-%d':
                return date_obj.strftime('%d %B %Y')
            elif fmt == '%Y-%m':
                return date_obj.strftime('%B %Y')
        except ValueError:
            continue
    return None


Train data successfully fetched from MongoDB

Test data successfully fetched from MongoDB



In [8]:
df= load_database()
test_id = "st_1155048"
test_article = df[df['st_id'] == test_id].reset_index(drop=True).to_dict()

{'_id': {0: ObjectId('6666ac3f6619e3e180cbbadf')},
 'Text': {0: 'BRUSSELS - European Council president Charles Michel said on Saturday that he had convened a video conference summit of European Union leaders on Tuesday to discuss the Hamas attacks on Israelis and Israel’s response.Mr Michel said the bloc stood in “full solidarity” with the people of Israel after the “brutal terrorist attacks” of a week ago.In an invitation letter to EU leaders, Mr Michel said Israel had the right to defend itself in compliance with international law.He said the siege of the Gaza Strip was raising alarm bells in the international community, prompting him to convene a video conference meeting on Tuesday at 5.30pm Central European Time (11.30pm Singapore time).“It is of utmost importance that the European Council, in line with the treaties and our values, sets our common position and establishes a clear unified course of action that reflects the complexity of the unfolding situation,” he wrote.Mr Michel s